<a href="https://colab.research.google.com/github/buddypia/openai-agents-sdk-colab/blob/master/multi_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Pythonパッケージインストール
!pip install openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

# ColabのシークレットからOpenAIのAPIキーを取得
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

# OpenAIエージェントにAPIキーを設定
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [36]:
import random

from agents import Agent, function_tool

# 乱数生成ツール
@function_tool
def random_number() -> int:
    """
    指定された最大値までの乱数を生成します。
    """
    return random.randint(0, 100)

# 1. 乱数を生成するエージェントを定義
random_number_agent = Agent(
    name="random_int_agent",
    instructions="あなたは乱数を生成します。",
    tools=[random_number],
)

In [37]:
from agents import Agent

# 2. 英語に翻訳するエージェントを定義
english_translate_agent = Agent(
    name="english_translate_agent",
    instructions="あなたは英語に翻訳します。",
)

In [55]:
from agents import Agent, WebSearchTool
from agents.model_settings import ModelSettings

# 3. サーチエージェントを定義
search_agent = Agent(
    name="Search agent",
    instructions="あなたはリサーチアシスタントです。検索語が与えられたら、ウェブを検索し、結果の簡潔な要約します",
    tools=[WebSearchTool()],
    model_settings=ModelSettings(tool_choice="required"),
)

In [56]:
from agents import Agent

# ユーザーのインプットに基づいて適切なエージェントに処理を引き渡す（ハンドオフ）エージェントを定義
triage_agent = Agent(
    name="triage_agent",
    instructions="ユーザーの入力に基づいて、適切なエージェントに引き渡してください。",
    handoffs=[random_number_agent, english_translate_agent, search_agent],
)

In [57]:
from agents import Runner, TResponseInputItem

# 最初のユーザーメッセージを取得
# 最初の担当エージェントを triage_agent に設定
agent = triage_agent

# ループしてエージェントを呼び出す
# 指示1: 乱数を生成して
# 指示2: 「休みの日は何をしていますか？」を英語に翻訳してください。
# 指示3: 2025年5月の最新ニュースを教えて
for attempt in range(3):
    query = input("こんにちは！何かお手伝いできることはありますか？: ")

    # 現在のエージェントを実行し、ストリーミングで結果を取得
    result = await Runner.run(
        agent, # 実行するエージェント
        query, # 入力データ
    )

    print(f"ハンドオフされたエージェント: {result.last_agent}")
    print(f"{result.final_output}\n")

こんにちは！何かお手伝いできることはありますか？: 乱数を生成して
ハンドオフされたエージェント: Agent(name='random_int_agent', instructions='あなたは乱数を生成します。', handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[FunctionTool(name='random_number', description='指定された最大値までの乱数を生成します。', params_json_schema={'properties': {}, 'title': 'random_number_args', 'type': 'object', 'additionalProperties': False, 'required': []}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x783f7fd88f40>, strict_json_schema=True)], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)
生成された乱数は 71 です。

